# Used for predicting ALSFRS_slope
see https://www.synapse.org/#!Synapse:syn2873386/wiki/ .
We assumed data is vectorized + clustered + 6 features were selected

In [1]:
from IPython.display import display

import pandas as pd
import numpy as np
import pickle
from sklearn import linear_model
from vectorizing_funcs import *
from modeling_funcs import *

## Revectorize the selected data
We now reload the metadata and the 6 attributes selected per cluster

In [2]:
all_feature_metadata = pickle.load( open('../all_feature_metadata.pickle', 'rb') )
train_data_means = pickle.load( open('../train_data_means.pickle', 'rb') )
train_data_std = pickle.load( open('../train_data_std.pickle', 'rb') )
best_features_per_cluster = pickle.load( open('../best_features_per_cluster.pickle', 'rb') )


df = pd.read_csv('../train_data_selected.csv', sep='|', index_col=False, dtype="unicode")
vectorized, _ = vectorize(df, all_feature_metadata)
normalized, _ = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)
print normalized.shape
normalized.head()

(1777, 225)


,family_ALS_hist_last,Hematocrit_pct_diff,Hematocrit_last,Hematocrit_mean_slope,Hematocrit_mean,Prothrombin Time (clotting)_pct_diff,Prothrombin Time (clotting)_last,Prothrombin Time (clotting)_mean_slope,Prothrombin Time (clotting)_mean,weight_pct_diff,...,Protein_mean_slope,Protein_mean,Uric Acid_pct_diff,Uric Acid_last,Uric Acid_mean_slope,Uric Acid_mean,ALT(SGPT)_pct_diff,ALT(SGPT)_last,ALT(SGPT)_mean_slope,ALT(SGPT)_mean
SubjectID,,,,,,,,,,,,,,,,,,,,,
533,0,0,0,0,0,0,0,0,0,0,...,0.033753,-0.767815,0,0,0,0,0.218664,-0.527906,0.050588,-0.738027
2956,0,0,0,0,0,0,0,0,0,0,...,0.318296,0.261546,0,0,0,0,-0.029158,-0.337255,-0.057131,-0.508923
4772,0,0,0,0,0,0,0,0,0,0,...,1.869789,-0.381805,0,0,0,0,0.388117,0.139374,-0.369608,-0.428063
5155,0,0,0,0,0,0,0,0,0,0,...,-0.334100,-1.089490,0,0,0,0,-0.151245,-0.337255,0.019108,-0.347202
7750,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0,0,0,0,-0.317810,-0.241929,-0.056664,-0.234896


In [3]:
slope = pd.read_csv('../train_slope.csv', sep = '|', index_col="SubjectID")
clusters = pd.read_csv('../train_kmeans_clusters.csv', sep = '|', index_col="SubjectID")

clusters.index = clusters.index.astype(str)
slope.index = slope.index.astype(str)
normalized.index = normalized.index.astype(str)

X = normalized.join(clusters)
Y = slope.join(clusters)

print Y.shape, X.shape, clusters.shape
display(X.head(3))
display(Y.head(3))


(1777, 2) (1777, 226) (1777, 1)


,family_ALS_hist_last,Hematocrit_pct_diff,Hematocrit_last,Hematocrit_mean_slope,Hematocrit_mean,Prothrombin Time (clotting)_pct_diff,Prothrombin Time (clotting)_last,Prothrombin Time (clotting)_mean_slope,Prothrombin Time (clotting)_mean,weight_pct_diff,...,Protein_mean,Uric Acid_pct_diff,Uric Acid_last,Uric Acid_mean_slope,Uric Acid_mean,ALT(SGPT)_pct_diff,ALT(SGPT)_last,ALT(SGPT)_mean_slope,ALT(SGPT)_mean,cluster
SubjectID,,,,,,,,,,,,,,,,,,,,,
533,0,0,0,0,0,0,0,0,0,0,...,-0.767815,0,0,0,0,0.218664,-0.527906,0.050588,-0.738027,0
2956,0,0,0,0,0,0,0,0,0,0,...,0.261546,0,0,0,0,-0.029158,-0.337255,-0.057131,-0.508923,0
4772,0,0,0,0,0,0,0,0,0,0,...,-0.381805,0,0,0,0,0.388117,0.139374,-0.369608,-0.428063,0


,ALSFRS_slope,cluster
SubjectID,,
533,-0.965608,0
649,-0.921717,1
1234,-0.914787,2


## Train a prediction model per cluster

In [4]:
model_per_cluster = get_model_per_cluster(X, Y)
    

cluster: 0 size: (620L,)
	 RMS error (0 is perfect): 0.63
	 explained variance score (1 is perfect): 0.01
cluster: 1 size: (621L,)
	 RMS error (0 is perfect): 0.66
	 explained variance score (1 is perfect): 0.01
cluster: 2 size: (536L,)
	 RMS error (0 is perfect): 0.58
	 explained variance score (1 is perfect): 0.02


In [5]:
with open("../model_per_cluster.pickle", "wb") as output_file:
    pickle.dump(model_per_cluster, output_file)


## Apply the model on both `train` and `test`

In [6]:

for t in ['train', 'test']:
    df = pd.read_csv('../' + t + '_data_selected.csv', sep='|', index_col=False)
    vectorized, _ = vectorize(df, all_feature_metadata)
    normalized, _ = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)
    
    clusters = pd.read_csv('../' + t + '_kmeans_clusters.csv', sep = '|', index_col=0)

    clusters.index = clusters.index.astype(str)
    normalized.index = normalized.index.astype(str)

    X = normalized.join(clusters)
    pred = X.apply(apply_model, args=[model_per_cluster], axis = 1)
    pred.to_csv('../' + t + '_prediction.csv',sep='|')

pred.head()

,cluster,prediction
SubjectID,,
750094,0,-0.780623
750148,0,-0.780623
751103,0,-0.780623
753073,0,-0.780623
753161,0,-0.780623
